# Scratchwork

In [1]:
from finite_algebras import *

In [2]:
psr3 = generate_powerset_ring(3)

In [3]:
psr3.about()


Ring: PSRing3
Description: Autogenerated Ring on powerset of {0,...,3} w/ symm. diff. (add) & intersection (mult)
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]


In [4]:
psr3.pprint()

Ring('PSRing3',
'Autogenerated Ring on powerset of {0,...,3} w/ symm. diff. (add) & intersection (mult)',
['{}', '{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}', '{0, 1, 2}'],

# Addition Table:
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
,
# Multiplication Table:
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1],
 [0, 0, 2, 0, 2, 0, 2, 2],
 [0, 0, 0, 3, 0, 3, 3, 3],
 [0, 1, 2, 0, 4, 1, 2, 4],
 [0, 1, 0, 3, 1, 5, 3, 5],
 [0, 0, 2, 3, 2, 3, 6, 6],
 [0, 1, 2, 3, 4, 5, 6, 7]]
)


In [5]:
psr2 = generate_powerset_ring(2)
psr2

Ring(
PSRing2,
Autogenerated Ring on powerset of {0,...,2} w/ symm. diff. (add) & intersection (mult),
['{}', '{0}', '{1}', '{0, 1}'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

In [6]:
psr2.pprint()

Ring('PSRing2',
'Autogenerated Ring on powerset of {0,...,2} w/ symm. diff. (add) & intersection (mult)',
['{}', '{0}', '{1}', '{0, 1}'],

# Addition Table:
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
,
# Multiplication Table:
[[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 2, 2], [0, 1, 2, 3]]
)


In [7]:
rng = Ring('Powerset Ring 2',

           'Ring on powerset of {0, 1}',

           # Elements must be strings
           ['{}', '{0}', '{1}', '{0, 1}'],

           # Addition table (symmetric difference of sets)
           [[0, 1, 2, 3],
            [1, 0, 3, 2],
            [2, 3, 0, 1],
            [3, 2, 1, 0]],

           # Multiplication table (intersection of sets)
           [[0, 0, 0, 0],
            [0, 1, 0, 1],
            [0, 0, 2, 2],
            [0, 1, 2, 3]]
          )

In [8]:
rng.pprint()

Ring('Powerset Ring 2',
'Ring on powerset of {0, 1}',
['{}', '{0}', '{1}', '{0, 1}'],

# Addition Table:
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
,
# Multiplication Table:
[[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 2, 2], [0, 1, 2, 3]]
)


In [9]:
rng.about()


Ring: Powerset Ring 2
Description: Ring on powerset of {0, 1}
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3  {0, 1}  {0, 1}       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


In [10]:
{1} ^ {0,1}

{0}

In [11]:
rng.add("{1}", "{0, 1}")

'{0}'

In [12]:
{1} & {0,1}

{1}

In [13]:
rng.mult("{1}", "{0, 1}")

'{1}'

In [15]:
rng.table

CayleyTable([[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]])

In [17]:
rng.add_table()

CayleyTable([[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]])

In [18]:
rng.mult_table()

CayleyTable([[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 2, 2], [0, 1, 2, 3]])

In [19]:
rng.mult()

3

In [23]:
set() & {1}

set()